In [49]:
import requests
from bs4 import BeautifulSoup

base_url = "https://pandas.pydata.org/docs/reference/index.html"
response = requests.get(base_url)

In [50]:
soup = BeautifulSoup(response.text, "lxml")
navbar = soup.find_all(attrs={"class": "reference internal"})

In [51]:
ref_url = "https://pandas.pydata.org/docs/reference/"
ref_url+ navbar[0]['href']

'https://pandas.pydata.org/docs/reference/io.html'

In [57]:
navbar[10].text

'Style'

In [52]:
first_level = {}
for nbar in navbar:
    text_val = nbar.text
    href = nbar['href']
    full_url = ref_url+ href
    if  "#" not in href:
        first_level.update({href:{"functions":[],"name":text_val,"url":full_url}})
    else:
        continue

In [53]:
for nbar in navbar:
    text_val = nbar.text
    href = nbar['href']
    full_url = ref_url+ href
    if "#" in href:
        parent_html = href.split("#")[0]
        if parent_html in first_level:
            first_level[parent_html]['functions'].append({"name":text_val,"url":full_url})
        else:
            print(text_val)

In [55]:
first_level['offset_frequency.html']['functions']
# https://pandas.pydata.org/docs/reference/io.html

[{'name': 'DateOffset',
  'url': 'https://pandas.pydata.org/docs/reference/offset_frequency.html#dateoffset'},
 {'name': 'BusinessDay',
  'url': 'https://pandas.pydata.org/docs/reference/offset_frequency.html#businessday'},
 {'name': 'BusinessHour',
  'url': 'https://pandas.pydata.org/docs/reference/offset_frequency.html#businesshour'},
 {'name': 'CustomBusinessDay',
  'url': 'https://pandas.pydata.org/docs/reference/offset_frequency.html#custombusinessday'},
 {'name': 'CustomBusinessHour',
  'url': 'https://pandas.pydata.org/docs/reference/offset_frequency.html#custombusinesshour'},
 {'name': 'MonthEnd',
  'url': 'https://pandas.pydata.org/docs/reference/offset_frequency.html#monthend'},
 {'name': 'MonthBegin',
  'url': 'https://pandas.pydata.org/docs/reference/offset_frequency.html#monthbegin'},
 {'name': 'BusinessMonthEnd',
  'url': 'https://pandas.pydata.org/docs/reference/offset_frequency.html#businessmonthend'},
 {'name': 'BusinessMonthBegin',
  'url': 'https://pandas.pydata.org/

## EXTRACTING FUNCTIONS

In [7]:
# parent_url = first_level['io.html']['url']

# parent_soup = BeautifulSoup(requests.get(parent_url).text,'lxml')
# url_id = "pickling"
# s1 = parent_soup.find(attrs={"id":url_id})
# s1

In [9]:
# first_level['io.html']['functions']

In [10]:
def get_links(id_elem,base_func_url,class_name,first_level_name,url_id):
    curr_urls = []
    try:
        func_urls = id_elem.find_all(attrs={"class":class_name})
        for odd_url in func_urls:
            try:
                func_url = odd_url.find('a')['href']
                curr_urls.append(base_func_url+func_url)
            except Exception as e:
                print(e)
                print(first_level_name)
                print(odd_url)
                print(url_id)
                print('-'*100)
        return curr_urls
    except Exception as e:
        print(e)
        func_urls = id_elem.find(attrs={"class":class_name}).find('a')['href']
        curr_urls.append(base_func_url+func_url)
        return curr_urls
    finally:
        return curr_urls
    
base_func_url = "https://pandas.pydata.org/docs/reference/"
for first_level_name in first_level:
    parent_url = first_level[first_level_name]['url']
    parent_soup = BeautifulSoup(requests.get(parent_url).text,'lxml')
    for idx,func in enumerate(first_level[first_level_name]['functions']):
        url_id = func['url'].split("#")[-1]
        id_elem = parent_soup.find(attrs={"id":url_id})
        odd_urls = get_links(id_elem,base_func_url,"row-odd",first_level_name,url_id)     
        even_urls = get_links(id_elem,base_func_url,"row-even",first_level_name,url_id)     
        # all_urls.extend(odd_urls)
        # all_urls.extend(even_urls)
        first_level[first_level_name]['functions'][idx]['function_urls'] = odd_urls + even_urls

'NoneType' object is not subscriptable
series.html
<tr class="row-odd"><th class="head"><p>Data Type</p></th>
<th class="head"><p>Accessor</p></th>
</tr>
accessors
----------------------------------------------------------------------------------------------------
'NoneType' object is not subscriptable
arrays.html
<tr class="row-odd"><th class="head"><p>Kind of Data</p></th>
<th class="head"><p>pandas Data Type</p></th>
<th class="head"><p>Scalar</p></th>
<th class="head"><p>Array</p></th>
</tr>
objects
----------------------------------------------------------------------------------------------------
'NoneType' object is not subscriptable
arrays.html
<tr class="row-odd"><th class="head"><p>PyArrow type</p></th>
<th class="head"><p>pandas extension type</p></th>
<th class="head"><p>NumPy type</p></th>
</tr>
objects
----------------------------------------------------------------------------------------------------


In [74]:
# first_level['io.html']['functions']

In [33]:
from tqdm import tqdm
import logging
from IPython.display import clear_output
logname = "pandas.log"
# logging.basicConfig(filename=logname,
#                     filemode='a',
#                     format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
#                     datefmt='%H:%M:%S',
#                     level=logging.DEBUG)

def get_param_data(first_level):
    not_worked = []
    for parent in first_level:
        parent_dict = first_level[parent]['functions']
        for idx,sub_level in enumerate(parent_dict):
            parent_dict[idx].update({"function_definitions":[]})
            for func_url in tqdm(sub_level['function_urls']):
                func_response = requests.get(func_url)
                func_soup = BeautifulSoup(func_response.content, "lxml",from_encoding="utf-8")
                
                func_name = func_soup.find("h1").text.replace("#","") #remove #
                elem = func_soup.find(attrs={"class":"sig sig-object py"})
                try:
                    full_function = elem.text.replace("[source]#","").replace("\n","")
                    func_text = func_soup.find("dd").find('p').text
                    curr_dict = {"function_name":func_name,"full_function":full_function,"function_text":func_text,"parameter_names_desc":[],"function_url":func_url}
                    em = func_soup.find_all(attrs={"class":"field-odd"})
                    if em[0].text =='Parameters:':
                        param_names = em[-1].find_all("dt")
                        desc_list = em[-1].find_all("dd")

                        # if len(param_names)!=len(desc_list):
                        #     print(func_name)
                        #     print(f"param_names={len(param_names)} and desc_list={len(desc_list)}")
                        # first_level[parent]['function_definitions'].append()
                        for pn,dn in zip(param_names,desc_list):
                            try:
                                param_name = pn.strong.text
                                param_type = pn.find(attrs={"class":"classifier"}).text
                                if param_name == "**kwargs": 
                                    continue
                                param_desc = dn.text
                                curr_dict['parameter_names_desc'].append({"param_name":param_name,"param_type":param_type,"param_desc":param_desc})
                            except Exception as e:
                                print(e,pn.text)
                    # else:
                    #     parent_dict[idx]['function_definitions'].append(curr_dict)
                    #     continue
                except Exception as e:
                    # print(e)
                    not_worked.append((func_name,func_text,e,func_url))
                    # logging.debug("With error {e} for {func_name} and {func_text}")
                finally:
                    parent_dict[idx]['function_definitions'].append(curr_dict)
        clear_output(wait=True)
    return first_level,not_worked

function_def_dict,not_worked = get_param_data(first_level)

100%|██████████| 1/1 [00:00<00:00,  1.39it/s]


In [63]:
not_worked[400]

('pandas.tseries.offsets.CustomBusinessMonthEnd.freqstr',
 'Return a string representing the frequency.',
 IndexError('list index out of range'),
 'https://pandas.pydata.org/docs/reference/api/pandas.tseries.offsets.CustomBusinessMonthEnd.freqstr.html#pandas.tseries.offsets.CustomBusinessMonthEnd.freqstr')

In [64]:
with open('not_worked.txt', 'w') as file:
    for item in not_worked:
        file.write(f"{item}\n")

In [37]:
import json
filename = "pandas_function_v3.json"

# Serialize the dictionary to a JSON string
json_data = json.dumps(function_def_dict, ensure_ascii=False)

# Write the JSON string to a file with UTF-8 encoding
with open(filename, "w", encoding="utf-8") as json_file:
    json_file.write(json_data)

print("Data has been successfully written to", filename)

Data has been successfully written to pandas_function_v3.json


In [87]:
import json
filename = "pandas_function_v2.json"

with open(filename, "r", encoding="utf-8") as json_file:
        # Load the JSON data from the file
        data = json.load(json_file)

In [67]:
func_url = 'https://pandas.pydata.org/docs/reference/api/pandas.read_clipboard.html'
# func_url = 'https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.nanoseconds.html'
func_response = requests.get(func_url)
func_soup = BeautifulSoup(func_response.content, "lxml",from_encoding="utf-8")

In [68]:
func_name = func_soup.find("h1").text.replace("#","") #remove #

In [69]:
func_name

'pandas.read_clipboard'

In [70]:
elem = func_soup.find(attrs={"class":"sig sig-object py"})
full_function = elem.text.replace("[source]#","").replace("\n","")
func_text = func_soup.find("dd").find('p').text

In [71]:
func_text

'Read text from clipboard and pass to read_csv().'

In [72]:
param_names = func_soup.find(attrs={"class":"field-list"})

In [73]:
em = param_names.find_all(attrs={"class":"field-odd"})

In [75]:
em[-1].text

'\nsepstr, default ‘\\s+’A string or regex delimiter. The default of \'\\\\s+\' denotes\none or more whitespace characters.\n\ndtype_backend{‘numpy_nullable’, ‘pyarrow’}, default ‘numpy_nullable’Back-end data type applied to the resultant DataFrame\n(still experimental). Behaviour is as follows:\n\n"numpy_nullable": returns nullable-dtype-backed DataFrame\n(default).\n"pyarrow": returns pyarrow-backed nullable ArrowDtype\nDataFrame.\n\n\nNew in version 2.0.\n\n\n**kwargsSee read_csv() for the full argument list.\n\n\n'

In [44]:
param_names

In [48]:
# len(em[-1].find_all('dt'))
em[-1].find_all('dt')

[<dt><strong>extra_args</strong><span class="classifier">list[str], default None</span></dt>,
 <dt><strong>run_doctests</strong><span class="classifier">bool, default False</span></dt>]

In [19]:
func_soup.find(attrs={"class":"field-list"}).find(attrs={"class":"classifier"})

<span class="classifier">str, default â\s+â</span>

In [20]:
em = func_soup.find_all(attrs={"class":"field-odd"})

In [21]:
em[-1].find_all("dt")

[<dt><strong>sep</strong><span class="classifier">str, default â\s+â</span></dt>,
 <dt><strong>dtype_backend</strong><span class="classifier">{ânumpy_nullableâ, âpyarrowâ}, default ânumpy_nullableâ</span></dt>,
 <dt><strong>**kwargs</strong></dt>]

In [22]:
for p in em[-1].find_all("p"):

    if p.find(attrs={"class":"versionmodified added"}) is None and p.find(attrs={"class":"versionmodified changed"}) is None:
        print(p.text)
        print('-'*100)

A string or regex delimiter. The default of '\\s+' denotes
one or more whitespace characters.
----------------------------------------------------------------------------------------------------
Back-end data type applied to the resultant DataFrame
(still experimental). Behaviour is as follows:
----------------------------------------------------------------------------------------------------
"numpy_nullable": returns nullable-dtype-backed DataFrame
(default).
----------------------------------------------------------------------------------------------------
"pyarrow": returns pyarrow-backed nullable ArrowDtype
DataFrame.
----------------------------------------------------------------------------------------------------
See read_csv() for the full argument list.
----------------------------------------------------------------------------------------------------


## CLEANING DATA

In [2]:
import json
filename = "pandas_function.json"

with open(filename, "r", encoding="utf-8") as json_file:
        # Load the JSON data from the file
        data = json.load(json_file)

In [6]:
len(data['io.html']['function_definitions'])

51

In [7]:
data['io.html']['function_definitions'][0]

{'function_name': 'pandas.read_pickle',
 'full_function': "pandas.read_pickle(filepath_or_buffer, compression='infer', storage_options=None)",
 'function_text': 'Load pickled pandas object (or any object) from file.',
 'parameter_names_desc': [{'param_name': 'filepath_or_bufferstr, path object, or file-like object',
   'param_desc': 'String, path object (implementing os.PathLike[str]), or file-like\nobject implementing a binary readlines() function.\nAlso accepts URL. URL is not limited to S3 and GCS.\n'},
  {'param_name': 'compressionstr or dict, default ‘infer’',
   'param_desc': "For on-the-fly decompression of on-disk data. If ‘infer’ and ‘filepath_or_buffer’ is\npath-like, then detect compression from the following extensions: ‘.gz’,\n‘.bz2’, ‘.zip’, ‘.xz’, ‘.zst’, ‘.tar’, ‘.tar.gz’, ‘.tar.xz’ or ‘.tar.bz2’\n(otherwise no compression).\nIf using ‘zip’ or ‘tar’, the ZIP file must contain only one data file to be read in.\nSet to None for no decompression.\nCan also be a dict with k

In [8]:
data['io.html']

{'functions': [{'name': 'Pickling',
   'url': 'https://pandas.pydata.org/docs/reference/io.html#pickling'},
  {'name': 'Flat file',
   'url': 'https://pandas.pydata.org/docs/reference/io.html#flat-file'},
  {'name': 'Clipboard',
   'url': 'https://pandas.pydata.org/docs/reference/io.html#clipboard'},
  {'name': 'Excel',
   'url': 'https://pandas.pydata.org/docs/reference/io.html#excel'},
  {'name': 'JSON',
   'url': 'https://pandas.pydata.org/docs/reference/io.html#json'},
  {'name': 'HTML',
   'url': 'https://pandas.pydata.org/docs/reference/io.html#html'},
  {'name': 'XML',
   'url': 'https://pandas.pydata.org/docs/reference/io.html#xml'},
  {'name': 'Latex',
   'url': 'https://pandas.pydata.org/docs/reference/io.html#latex'},
  {'name': 'HDFStore: PyTables (HDF5)',
   'url': 'https://pandas.pydata.org/docs/reference/io.html#hdfstore-pytables-hdf5'},
  {'name': 'Feather',
   'url': 'https://pandas.pydata.org/docs/reference/io.html#feather'},
  {'name': 'Parquet',
   'url': 'https://p

In [66]:
function_def_dict['arrays.html']['functions'][0]

{'name': 'Objects',
 'url': 'https://pandas.pydata.org/docs/reference/arrays.html#objects',
 'function_urls': ['https://pandas.pydata.org/docs/reference/api/pandas.Timedelta.html#pandas.Timedelta',
  'https://pandas.pydata.org/docs/reference/api/pandas.IntervalDtype.html#pandas.IntervalDtype',
  'https://pandas.pydata.org/docs/reference/api/pandas.Float64Dtype.html#pandas.Float64Dtype',
  'https://pandas.pydata.org/docs/reference/api/pandas.SparseDtype.html#pandas.SparseDtype',
  'https://pandas.pydata.org/docs/reference/api/pandas.BooleanDtype.html#pandas.BooleanDtype',
  'https://pandas.pydata.org/docs/reference/api/pandas.array.html#pandas.array',
  'https://pandas.pydata.org/docs/reference/https://arrow.apache.org/docs/python/generated/pyarrow.int8.html#pyarrow.int8',
  'https://pandas.pydata.org/docs/reference/https://arrow.apache.org/docs/python/generated/pyarrow.int32.html#pyarrow.int32',
  'https://pandas.pydata.org/docs/reference/https://arrow.apache.org/docs/python/generated/